In [ ]:
import matplotlib.pyplot as plt
%matplotlib inline
from astropy.time import Time, TimeDelta
import numpy as np
import asyncio
from lsst.summit.utils.tmaUtils import TMAEventMaker, plotEvent, TMAEvent, getAzimuthElevationDataForEvent

from lsst.summit.utils.efdUtils import makeEfdClient
client = makeEfdClient()

In [ ]:
dayObs = 20231123
eventMaker = TMAEventMaker()
events = eventMaker.getEvents(dayObs)
tracks = [e for e in events if e.type.name=="TRACKING"]


In [ ]:
%matplotlib inline
seqNum = 351
fig = plt.figure(figsize=(10, 8))
plotEvent(client, events[seqNum], fig, doFilterResiduals=True)
plt.savefig(f"/home/c/cslage/u/MTMount/mount_plots/RubinTV_Tracking_Plot_MTMount_Filtered_{dayObs}_{seqNum}.png")

In [ ]:
seqNum = 257
event = events[seqNum]
azimuthData, elevationData = getAzimuthElevationDataForEvent(
                    client, event, doFilterResiduals=True, postPadding=-0.2)

In [ ]:
azError = azimuthData['azError']
elError = elevationData['elError']


N = 10
azRolling = azError.rolling(N).sum() / N
elRolling = elError.rolling(N).sum() / N
goodAzRolling = azRolling[abs(azRolling) < 0.01]
goodElRolling = elRolling[abs(elRolling) < 0.01]
if (len(goodAzRolling) == 0) or (len(goodElRolling) == 0):
    timeSettled = event.end
else:
    badAzRolling = azRolling[abs(azRolling) > 0.01]
    badElRolling = elRolling[abs(elRolling) > 0.01]
    if len(badElRolling) == 0:
        timeSettledEl = event.begin
    else:
        timeSettledEl = Time(badElRolling.index[-1])
    if len(badAzRolling) == 0:
        timeSettledAz = event.begin
    else:
        timeSettledAz = Time(badAzRolling.index[-1])
    timeSettled = max(timeSettledEl, timeSettledAz)
timeSettled

In [ ]:
elRolling.plot()
azRolling.plot()
plt.axvline(timeSettled.isot)
plt.axhline(-0.01, ls='--', color='black')
plt.axhline(0.01, ls='--', color='black')

In [ ]:

fig = plt.figure(figsize=(10, 8))
plotEvent(client, events[seqNum], fig, azimuthData=azimuthData, 
                      elevationData=elevationData, postPadding=-0.1)
ax = fig.get_axes()[1]
ax.axvline(event.begin.utc.datetime, ls='--', color='black')
ax.axvline(timeSettled.utc.datetime, ls='--', color='green')
ax.axhline(0.02, color='red')
#plt.savefig(f"/home/c/cslage/u/MTMount/mount_plots/RubinTV_Tracking_Plot_MTMount_Filtered_{dayObs}_{seqNum}.png")

In [ ]:
ax = fig.get_axes()[1]

In [ ]:
dayObs = 20240212
events = eventMaker.getEvents(dayObs)
tracks = [e for e in events if e.type.name=="TRACKING"]

In [ ]:
len(tracks)

In [ ]:
seqNum = 123
event = events[seqNum]
fig = plt.figure(figsize=(10, 8))
plotEvent(client, event, fig)
plt.savefig(f"/home/c/cslage/u/MTMount/mount_plots/RubinTV_Tracking_Plot_MTMount_{dayObs}_{seqNum}.png")

In [ ]:
from lsst.summit.utils.efdUtils import (getEfdData,
                       makeEfdClient,
                       efdTimestampToAstropy,
                       COMMAND_ALIASES,
                       getDayObsForTime,
                       getDayObsStartTime,
                       getDayObsEndTime,
                       )

In [ ]:
prePadding = 0.0; postPadding = 0.5

azimuthData = getEfdData(client,
                         'lsst.sal.MTMount.azimuth',
                         event=event,
                         prePadding=prePadding,
                         postPadding=postPadding)
elevationData = getEfdData(client,
                           'lsst.sal.MTMount.elevation',
                           event=event,
                           prePadding=prePadding,
                           postPadding=postPadding)
if event.type.name == 'TRACKING':
    pointingData = getEfdData(client,
                              'lsst.sal.MTPtg.currentTargetStatus',
                              event=event,
                              prePadding=prePadding,
                              postPadding=postPadding)


In [ ]:
azimuthData.columns

In [ ]:
from astropy.time import Time, TimeDelta
az_times = azimuthData['timestamp'].values
el_times = elevationData['timestamp'].values
ptg_times = pointingData['timestamp'].values
ptg_az_values = pointingData['demandAz'].values
ptg_el_values = pointingData['demandEl'].values
az_values = azimuthData['actualPosition'].values
el_values = elevationData['actualPosition'].values
az_demand_interp = np.interp(az_times, ptg_times, ptg_az_values)
el_demand_interp = np.interp(el_times, ptg_times, ptg_el_values)
az_error = (az_values - az_demand_interp) * 3600
el_error = (el_values - el_demand_interp) * 3600


In [ ]:
%matplotlib inline
plt.title("Azimuth MTPtg 2024-02-12, seqNum 123")
pointingData['demandAz'].plot(marker='x', label='Demand')
azimuthData['actualPosition'].plot(marker='o', label='Measured')
plt.axvline(event.end.isot, ls='--', color='black', label="event.end")
plotStart = Time("2024-02-12T23:14:10", scale='utc')
plotEnd = Time("2024-02-12T23:14:12", scale='utc')
plt.xlim(plotStart.isot, plotEnd.isot) 
#plt.ylim(-9.975, -9.925)
plt.ylabel("Azimuth (deg)")
plt.xlabel("Time(200ms steps)")
plt.legend()
plt.savefig(f"/home/c/cslage/u/MTMount/mount_plots/RubinTV_MTPtg_{dayObs}_{seqNum}.png")

In [ ]:
%matplotlib inline
plt.title("Azimuth MTPtg 2024-02-12, seqNum 123")
azimuthData['demandPosition'].plot(marker='x', label='Demand')
azimuthData['actualPosition'].plot(marker='o', label='Measured')
plt.axvline(event.end.isot, ls='--', color='black', label="event.end")
plotStart = Time("2024-02-12T23:14:10", scale='utc')
plotEnd = Time("2024-02-12T23:14:12", scale='utc')
plt.xlim(plotStart.isot, plotEnd.isot) 
plt.ylim(-9.975, -9.925)
plt.ylabel("Azimuth (deg)")
plt.xlabel("Time")
plt.legend()
plt.savefig(f"/home/c/cslage/u/MTMount/mount_plots/RubinTV_demandPosition_BlowUp_{dayObs}_{seqNum}.png")